In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

In [5]:
data_path = Path("data_set/housing.csv")
df = pd.read_csv(data_path)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [7]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [30]:
X = df.drop("median_house_value",axis=1)
y = df["median_house_value"]

In [31]:
num_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',]
cat_columns = ['ocean_proximity']

## preprocessing 

create a pipeline for numerical columns
1. handle missing values
2. scaling

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [33]:
num_pipeline = Pipeline([
    ("numeric_imputer",SimpleImputer(strategy='median')),
    ("num_scalar",StandardScaler()),
])

In [34]:
# i don't want to give names, i want model gives

In [35]:
from sklearn.pipeline import make_pipeline

In [36]:
num_pipeline = make_pipeline(SimpleImputer(strategy='median'),StandardScaler())

In [37]:
num_pipeline

,steps,"[('simpleimputer', ...), ('standardscaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [38]:
num_pipeline.named_steps

{'simpleimputer': SimpleImputer(strategy='median'),
 'standardscaler': StandardScaler()}

create a pipeline for categorical columns
1. handle missing values
2. Onehotencoding

In [39]:
from sklearn.preprocessing import OneHotEncoder

In [40]:
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"),
                             OneHotEncoder(handle_unknown='ignore'))

In [41]:
cat_pipeline.named_steps

{'simpleimputer': SimpleImputer(strategy='most_frequent'),
 'onehotencoder': OneHotEncoder(handle_unknown='ignore')}

### combine numeric and categorical

In [42]:
from sklearn.compose import ColumnTransformer

In [43]:
preprocessing = ColumnTransformer([
    ("numeric",num_pipeline,num_columns),
    ("categorical",cat_pipeline,cat_columns),
])

In [44]:
preprocessing

,transformers,"[('numeric', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


### add model

In [45]:
from sklearn.linear_model import LinearRegression

In [46]:
model = Pipeline([
    ("preprocessing",preprocessing),
    ("lin_model",LinearRegression()),
])

In [47]:
model

,steps,"[('preprocessing', ...), ('lin_model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [49]:
model.fit(X,y)

,steps,"[('preprocessing', ...), ('lin_model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numeric', ...), ('categorical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [50]:
model.predict(X.iloc[:5])

array([408492.35829822, 423996.66388553, 378466.63041436, 321115.43273007,
       255844.57608296])

In [51]:
y[:5]

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: median_house_value, dtype: float64